# Investigation: AWS Privilege Escalation Using IAM Login Profile

This notebook assists in investigating this detection.

In [ ]:
from scanner_client import ScannerClient
import pandas as pd
import plotly.express as px

# Initialize Scanner Client
client = ScannerClient(api_url="https://api.kast-apse1.scanner.dev", api_key="YOUR_API_KEY")

In [ ]:
query = """
%ingest.source_type="aws:cloudtrail"
eventSource:("iam.amazonaws.com" "signin.amazonaws.com")
(
  (
    eventSource:"iam.amazonaws.com"
    eventName:("CreateLoginProfile" "UpdateLoginProfile")
  )
  or
  (
    eventSource:"signin.amazonaws.com"
    eventName:"ConsoleLogin"
    responseElements.ConsoleLogin:"Success"
  )
)
not errorCode:*
| groupbycount(
    recipientAccountId,
    sourceIPAddress
  )
| where @q.count > 1
"""

results = client.query(query, time_range_s=86400)
df = pd.DataFrame(results)
df.head()